# Lab 1 (YouTube Videos Summarizer)

A Demo for summarization app utilizing youtube transcript API and Bart summary generator model

Methodology:
1. Extracting the video id from a provided input link
2. Transcript is extracted using pre-built function]
3. Providing Bart with the transcript for summarization

---

## Installing Required Packages

In [1]:
!pip install transformers==4.52.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 82.1 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 30.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.


In [2]:
!pip install youtube-transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.1/485.1 kB 7.8 MB/s eta 0:00:00:00:01


---

## HuggingFace Connection

For Retrieving Models

In [3]:
from huggingface_hub import login
login()

---

## Defining Extraction Function

In [4]:
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

In [5]:
def extract_video_id(url: str) -> str:
    """
    Extract the YouTube video ID from a URL.
    Raises ValueError if no 'v' parameter is found.
    """
    parsed = urlparse(url)
    qs = parse_qs(parsed.query)
    video_ids = qs.get('v')
    
    if not video_ids:
        raise ValueError(f"No video id found in URL: {url}")
    return video_ids[0]

---

## Calling Bart (Summariztion Model)

In [6]:
from transformers import pipeline, set_seed

2025-10-21 17:59:42.706955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761069583.100397      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761069583.220459      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
# setting randomization
set_seed(42)

In [8]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [9]:
# Defining configs
bart_config = {
    "max_length": 472,
    "min_length": 200,
    "do_sample": False,
    "length_penalty": 2.0
}

---

## Trying the Model

In [10]:
video_link = "https://www.youtube.com/watch?v=rHLEWRxRGiM&list=TLPQMjExMDIwMjXbhHt8H4YZWQ&index=3"

In [11]:
video_id = extract_video_id(video_link)

In [12]:
API = YouTubeTranscriptApi()
fetched_transcript = API.fetch(video_id, languages=['en'])
text = "\n".join(snippet.text for snippet in fetched_transcript)

In [13]:
response = summarizer(text, **bart_config)

summary = response[0].get('summary_text')

In [14]:
print(summary)

In the last two videos I talked about linear transformations and matrices. In the next video I'll start getting into the determinant. In general throughout the series we'll work mainly in two dimensions. But once you get all the core ideas in two, they carry over pretty seamlessly to higher dimensions. It turns out that 3D matrix multiplication is actually pretty important for fields  like computer graphics and robotics, since things like rotations and three dimensions are hard to describe but easier to wrap your mind around if you break them down into separate, easier-to-think-about rotations. In fact, a good way to test your understanding of this is to try to reason through what specifically this matrix multiplication should look like, and how it relates to the idea of applying successive transformations in space in 2D. The next video will be about determinant matrix multiplying in 2D and 3D, and the next will be on determinants in 3D and 4D.


---

## Encapsulating Usage in a method

In [15]:
def summarize_youtube_video(video_link, summarizer, bart_config):
    """
    Extracts transcript text from a YouTube video and summarizes it.

    Args:
        video_link (str): The full YouTube video URL.
        summarizer (callable): A summarization function or model pipeline (e.g., from transformers).
        bart_config (dict): Configuration parameters for the summarizer.

    Returns:
        str: The generated summary text.
    """
    # Extract video ID
    video_id = extract_video_id(video_link)

    # Fetch transcript
    API = YouTubeTranscriptApi()
    fetched_transcript = API.fetch(video_id, languages=['en'])

    # Combine transcript text
    text = "\n".join(snippet.text for snippet in fetched_transcript)

    # Generate summary
    response = summarizer(text, **bart_config)
    summary = response[0].get('summary_text')

    return summary

In [16]:
# Try
print(summarize_youtube_video(video_link, summarizer, bart_config))

In the last two videos I talked about linear transformations and matrices. In the next video I'll start getting into the determinant. In general throughout the series we'll work mainly in two dimensions. But once you get all the core ideas in two, they carry over pretty seamlessly to higher dimensions. It turns out that 3D matrix multiplication is actually pretty important for fields  like computer graphics and robotics, since things like rotations and three dimensions are hard to describe but easier to wrap your mind around if you break them down into separate, easier-to-think-about rotations. In fact, a good way to test your understanding of this is to try to reason through what specifically this matrix multiplication should look like, and how it relates to the idea of applying successive transformations in space in 2D. The next video will be about determinant matrix multiplying in 2D and 3D, and the next will be on determinants in 3D and 4D.


---

## Handling Long Videos Issue

### 1. Defining a function that split long transcript into chunks

In [17]:
def chunk_text(text, chunk_size=2000, overlap=100):
    """Splits long text into chunks under the token limit."""
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        print(end-start)
        start += chunk_size - overlap
    return chunks

### 2. New summarization function

In [18]:
def summarize_long_youtube_video(video_link, summarizer, bart_config):
    """
    Fetches YouTube transcript and summarizes it in chunks to handle long videos.
    """
    # Extract video ID
    video_id = extract_video_id(video_link)

    # YouTube Transcript API
    API = YouTubeTranscriptApi()
    
    # Fetch transcript
    fetched_transcript = API.fetch(video_id, languages=['en'])

    # Combine transcript text
    text = " ".join([snippet.text for snippet in fetched_transcript])

    # Split into chunks
    chunks = chunk_text(text)

    # Summarize each chunk
    partial_summaries = []
    for chunk in chunks:
        response = summarizer(chunk, **bart_config)
        partial_summaries.append(response[0]['summary_text'])

    # Combine and summarize again (hierarchical)
    combined_summary_text = " ".join(partial_summaries)
    if len(combined_summary_text) > 1000:
        final_summary = summarizer(combined_summary_text, **bart_config)[0]['summary_text']
    else:
        final_summary = combined_summary_text

    return final_summary

In [19]:
# Try
print(summarize_long_youtube_video("https://www.youtube.com/watch?v=VqrYlDcZQ54", summarizer, bart_config))

Your max_length is set to 472, but your input_length is only 397. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=198)


2000
2000
2000
2000
2000
2000
2000
2000
2000


Your max_length is set to 472, but your input_length is only 431. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=215)
Your max_length is set to 472, but your input_length is only 427. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=213)
Your max_length is set to 472, but your input_length is only 422. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=211)
Your max_length is set to 472, but your input_length is only 410. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
